# Dynamic Vertical Triplet Energy Benchmarking (DvTEBench)

## Import Dependencies

In [2]:
from pathlib import Path
import csv, math
import pandas as pd

import numpy as np
from numpy.random import seed
from numpy.random import randn

from scipy.stats import norm
from scipy.stats import t
from scipy.optimize import curve_fit

from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.mixture import GaussianMixture

## Load Datased and Experimental Values

Provide the names of the molecules you want to be included in the benchmark in the "species" list. Note: make sure that the names are consistent with the names of string before the underscore "_" delimiter in the output.csv files. Additionally the .csv files should be located in the same working directory.

Provide the experimental triplet energy value in "exptl_values" list.

In [3]:
# List of molecules
species = ['Naphthalene', 'Benzene', 'Dimethyl-Diazene', 'Teramethylethene', 'Isoprene', 'Azulene', 
           'Trans-Stilbene', 'Biphenyl', 'Cis-Stilbene', 'Styrene', 'Norbornene', 'Indene', 'Cyclopentene', 
           'Cyclohexenone', 'Cyclohexene', 'Trans-b-Methylstyrene', '1,1-Dichloroethene', 'Ethene', 
           'Cis-b-Methylstyrene', 'Diphenylacetylene']

# List of experimentally determined triplet energies (kcal/mol)
exptl_values = [60.5, 83.6, 54.0, 75.8, 60.0, 39.0, 51.0, 69.5, 55.5, 60.8, 72.3, 64.1, 79.4, 67.0, 
                81.2, 60.5, 73.3, 84.0, 66.5, 62.7]

# CSV outputs containing DFT data
output_list = [f"{entry}_output.csv" for entry in species]

mol_list = zip(output_list, species, exptl_values)

# Normal Distribution Models

This code utilises the compiled .csv dataset of vertical triplet energies generated using DvTEProc (i.e. {MoleculeName}_output.csv). Then using the mean and standard deviation from fitted normla distributions it construct cumulative distribution functions for each species. It then stores predicted triplet energies using a range of populations starting from 0.001 to 0.050 in steps of 0.001 as a new .csv file called "df_proc_normal.csv". The code then proceed to construct linear regressions of predicted triplet energy against experimental values for all selected populations within the 0.001 and 0.050 range, followed by reporting all R2s, MAEs and RMSDs. The model with the lowest R2 is then printed out at the end.

In [4]:
# Get the current working directory
current_directory = Path.cwd()

# Create an empty dataframe df_proc
df_proc = pd.DataFrame()

# Add species names
df_proc['Name'] = species

# Define the range of desired populations
desired_populations = np.arange(0.001, 0.051, 0.001)    #Modify here if you whish to change the range

# Add columns to df_proc for each desired population
for pop in desired_populations:
    df_proc[f'{pop:.3f}'] = np.nan

# Add experimental values
df_proc['Experimental'] = exptl_values

# Loop over the CSV filenames
for file, name in zip(output_list,species):
    
    # Construct the full path to the CSV file in the current directory
    csv_path = current_directory / file
    df = pd.read_csv(csv_path)
    
    # Calculate mean and standard deviation
    mu = df['dE'].mean()
    sigma = df['dE'].std()

    # Loop over desired populations
    for pop in desired_populations:
        # Find corresponding x value for desired y value
        x_value = norm.ppf(pop, loc=mu, scale=sigma)

        # Round x_value to 1 decimal place
        x_value = round(x_value, 1)

        # Update df_proc with the calculated x_value
        df_proc.at[df_proc[df_proc['Name'] == name].index[0], f'{pop:.3f}'] = x_value
        
        
# Save dataframe of compiled data as .csv
df_proc.to_csv('df_proc_normal.csv', index=False)

# Extract the 'Experimental' column
exptl_values = df_proc['Experimental'].values

# Initialize variables to keep track of the column with the highest R2
max_R2 = 0
best_column = None

# Display the updated dataframe of compiled data
print(df_proc)

for column in df_proc.columns[1:-1]:
    # Extract the column values
    column_values = df_proc[column].values

    # Remove NaN values from both arrays
    mask = ~np.isnan(exptl_values) & ~np.isnan(column_values)
    experimental_values_cleaned = exptl_values[mask]
    column_values_cleaned = column_values[mask]

    # Calculate Mean Squared Error
    mse = float("{:.1f}".format(mean_squared_error(experimental_values_cleaned, column_values_cleaned)))

    # Calculate RMSD
    rmsd = float("{:.1f}".format(math.sqrt(mse)))

    # Calculate Mean Absolute Error
    mae = float("{:.1f}".format(mean_absolute_error(experimental_values_cleaned, column_values_cleaned)))

    # Calculate R2
    r2 = float("{:.2f}".format(r2_score(experimental_values_cleaned, column_values_cleaned)))
    
    # Check if the current column has the highest R2
    if r2 >= max_R2:
        max_R2 = r2
        best_column = column

    # Print results
    print(f"Population: {column}")
    print(f"R2: {r2:.2f}")
    print(f"RMSD: {rmsd:.1f}")
    print(f"MAE: {mae:.1f}")
    print("--------------------")

# Print the column with the highest R2
print(f"Best Model Population: {best_column}")
print(f"Highest R2: {max_R2:.2f}")

                     Name  0.001  0.002  0.003  0.004  0.005  0.006  0.007  \
0             Naphthalene   57.7   59.0   59.9   60.5   61.0   61.4   61.8   
1                 Benzene   76.7   78.1   79.0   79.6   80.1   80.5   80.9   
2        Dimethyl-Diazene   48.8   49.7   50.3   50.7   51.0   51.2   51.5   
3        Teramethylethene   72.7   74.4   75.4   76.2   76.8   77.3   77.8   
4                Isoprene   51.9   53.9   55.1   56.0   56.7   57.3   57.8   
5                 Azulene   33.5   34.5   35.1   35.5   35.9   36.2   36.5   
6          Trans-Stilbene   45.8   47.6   48.7   49.5   50.2   50.7   51.2   
7                Biphenyl   67.5   69.1   70.1   70.8   71.4   71.9   72.3   
8            Cis-Stilbene   52.9   54.8   55.9   56.8   57.5   58.0   58.5   
9                 Styrene   58.0   59.8   60.9   61.7   62.4   62.9   63.4   
10             Norbornene   68.6   70.5   71.6   72.5   73.2   73.7   74.2   
11                 Indene   58.3   60.0   61.1   61.9   62.5   6

# 2GMM Models

Similar to the code above, this cell tries to fit a Guassian Mixture Model (GMM) to the dynamics data utilizing two guassians, which will then be used to construct the cumulative distribution function to compile predicted triplet energies using a range of populations. The code then proceed to construct linear regressions of predicted triplet energy against experimental values for all selected populations within the 0.001 and 0.050 range, followed by reporting all R2s, MAEs and RMSDs. The model with the lowest RMSD is then printed out at the end.

In [5]:
# Get the current working directory
current_directory = Path.cwd()

# Create an empty dataframe df_proc
df_proc = pd.DataFrame()

# Add species names
df_proc['Name'] = species

# Define the range of desired populations
desired_populations = np.arange(0.001, 0.051, 0.001)

# Add columns to df_proc for each desired population
for pop in desired_populations:
    df_proc[f'{pop:.3f}'] = np.nan

# Add experimental values
df_proc['Experimental'] = exptl_values

# Loop over the CSV filenames
for file, name in zip(output_list,species):

    # Construct the full path to the CSV file in the current directory
    csv_path = current_directory / file
    df = pd.read_csv(csv_path)

    # Extract the data as a 1D array
    data = df['dE'].values

    # Create a Gaussian Mixture Model with 2 components
    model = GaussianMixture(n_components=2)

    # Fit the model to the data
    model.fit(data.reshape(-1, 1))

    # Get the parameters of the fitted distributions
    means = model.means_
    covariances = model.covariances_
    weights = model.weights_

    # Generate data points for plotting the distributions
    x = np.linspace(data.min(), data.max(), 1000).reshape(-1, 1)
    pdf1 = weights[0] * np.exp(-(x - means[0])**2 / (2 * covariances[0])) / np.sqrt(2 * np.pi * covariances[0])
    pdf2 = weights[1] * np.exp(-(x - means[1])**2 / (2 * covariances[1])) / np.sqrt(2 * np.pi * covariances[1])

    # Sum of the two distributions
    pdf_sum = pdf1 + pdf2
    cdf = np.cumsum(pdf_sum) * (x[1] - x[0])

    # Loop over desired populations
    for pop in desired_populations:
        # Find corresponding x value for desired y value
        x_value = np.interp(pop, cdf, x.flatten())

        # Update df_proc with the calculated x_value
        df_proc.at[df_proc[df_proc['Name'] == name].index[0], f'{pop:.3f}'] = x_value

# Assuming df_proc is your dataframe
df_proc.to_csv('df_proc_2GMM.csv', index=False)

# Extract the 'Experimental' column
experimental_values = df_proc['Experimental'].values

# Initialize variables to keep track of the column with the lowest MAE
max_R2 = 0
best_column = None

# Display the updated df_proc
print(df_proc)

# Loop over columns in df_proc (excluding 'Name' and 'Experimental' columns)
for column in df_proc.columns[1:-1]:
    # Extract the column values
    column_values = df_proc[column].values

    # Remove NaN values from both arrays
    mask = ~np.isnan(exptl_values) & ~np.isnan(column_values)
    experimental_values_cleaned = exptl_values[mask]
    column_values_cleaned = column_values[mask]

    # Calculate Mean Squared Error
    mse = float("{:.1f}".format(mean_squared_error(experimental_values_cleaned, column_values_cleaned)))

    # Calculate RMSD
    rmsd = float("{:.1f}".format(math.sqrt(mse)))

    # Calculate Mean Absolute Error
    mae = float("{:.1f}".format(mean_absolute_error(experimental_values_cleaned, column_values_cleaned)))

    # Calculate R-squared
    r2 = float("{:.2f}".format(r2_score(experimental_values_cleaned, column_values_cleaned)))
    
    # Check if the current column has a lower MAE
    if r2 >= max_R2:
        max_R2 = r2
        best_column = column

    # Print results
    print(f"Population: {column}")
    print(f"R2: {r2:.2f}")
    print(f"RMSD: {rmsd:.1f}")
    print(f"MAE: {mae:.1f}")
    print("--------------------")

# Print the column with the lowest MAE
print(f"Best Model Population: {best_column}")
print(f"Highest R2: {max_R2:.2f}")

                     Name      0.001      0.002      0.003      0.004  \
0             Naphthalene  60.148007  60.998128  61.575291  62.018149   
1                 Benzene  76.544087  77.761268  78.629328  79.307832   
2        Dimethyl-Diazene  49.673046  50.312860  50.734679  51.054163   
3        Teramethylethene  77.893585  78.498678  78.973302  79.366384   
4                Isoprene  56.315530  57.406916  58.165665  58.754483   
5                 Azulene  34.030124  34.898994  35.439553  35.839020   
6          Trans-Stilbene  49.455850  50.395439  51.083805  51.632182   
7                Biphenyl  70.221579  71.458355  72.233112  72.806914   
8            Cis-Stilbene  56.465496  57.921557  58.823618  59.488960   
9                 Styrene  60.111502  61.561774  62.460855  63.124494   
10             Norbornene  73.393166  74.276728  74.915249  75.420522   
11                 Indene  59.283790  60.683913  61.577399  62.244587   
12           Cyclopentene  76.707081  78.087070  78

# 3GMM Models

Same as above but using 3 gaussian functions

In [6]:
# Get the current working directory
current_directory = Path.cwd()

# Create an empty dataframe df_proc
df_proc = pd.DataFrame()

# Add species names
df_proc['Name'] = species

# Define the range of desired populations
desired_populations = np.arange(0.001, 0.051, 0.001)

# Add columns to df_proc for each desired population
for pop in desired_populations:
    df_proc[f'{pop:.3f}'] = np.nan

# Add experimental values
df_proc['Experimental'] = exptl_values

# Loop over the CSV filenames
for csv_filename in output_list:
    # Construct the full path to the CSV file in the current directory
    csv_path = current_directory / csv_filename
    
    df = pd.read_csv(csv_path)

    # Extract the file name without "_output.csv"
    file_name = csv_filename.replace("_output.csv", "")

    # Extract the data as a 1D array
    data = df['dE'].values

    # Create a Gaussian Mixture Model with 2 components
    model = GaussianMixture(n_components=3)

    # Fit the model to the data
    model.fit(data.reshape(-1, 1))

    # Get the parameters of the fitted distributions
    means = model.means_
    covariances = model.covariances_
    weights = model.weights_

    # Generate data points for plotting the distributions
    x = np.linspace(data.min(), data.max(), 1000).reshape(-1, 1)
    pdf1 = weights[0] * np.exp(-(x - means[0])**2 / (2 * covariances[0])) / np.sqrt(2 * np.pi * covariances[0])
    pdf2 = weights[1] * np.exp(-(x - means[1])**2 / (2 * covariances[1])) / np.sqrt(2 * np.pi * covariances[1])
    pdf3 = weights[2] * np.exp(-(x - means[2])**2 / (2 * covariances[2])) / np.sqrt(2 * np.pi * covariances[2])
    
    # Sum of the two distributions
    pdf_sum = pdf1 + pdf2 + pdf3
    cdf = np.cumsum(pdf_sum) * (x[1] - x[0])

    # Loop over desired populations
    for pop in desired_populations:
        # Find corresponding x value for desired y value
        x_value = np.interp(pop, cdf, x.flatten())

        # Update df_proc with the calculated x_value
        df_proc.at[df_proc[df_proc['Name'] == file_name].index[0], f'{pop:.3f}'] = x_value

# Assuming df_proc is your dataframe
df_proc.to_csv('df_proc_3GMM.csv', index=False)

# Extract the 'Experimental' column
experimental_values = df_proc['Experimental'].values

# Initialize variables to keep track of the column with the lowest MAE
max_R2 = 0
best_column = None

# Display the updated df_proc
print(df_proc)

# Loop over columns in df_proc (excluding 'Name' and 'Experimental' columns)
for column in df_proc.columns[1:-1]:
    # Extract the column values
    column_values = df_proc[column].values

    # Remove NaN values from both arrays
    mask = ~np.isnan(exptl_values) & ~np.isnan(column_values)
    experimental_values_cleaned = exptl_values[mask]
    column_values_cleaned = column_values[mask]

    # Calculate Mean Squared Error
    mse = float("{:.1f}".format(mean_squared_error(experimental_values_cleaned, column_values_cleaned)))

    # Calculate RMSD
    rmsd = float("{:.1f}".format(math.sqrt(mse)))

    # Calculate Mean Absolute Error
    mae = float("{:.1f}".format(mean_absolute_error(experimental_values_cleaned, column_values_cleaned)))

    # Calculate R-squared
    r2 = float("{:.2f}".format(r2_score(experimental_values_cleaned, column_values_cleaned)))
    
    # Check if the current column has a lower MAE
    if r2 >= max_R2:
        max_R2 = r2
        best_column = column

    # Print results
    print(f"Population: {column}")
    print(f"R2: {r2:.2f}")
    print(f"RMSD: {rmsd:.1f}")
    print(f"MAE: {mae:.1f}")
    print("--------------------")

# Print the column with the lowest MAE
print(f"Best Model Population: {best_column}")
print(f"Highest R2: {max_R2:.2f}")

                     Name      0.001      0.002      0.003      0.004  \
0             Naphthalene  60.374309  61.202693  61.750696  62.166792   
1                 Benzene  77.496269  78.737249  79.531549  80.121127   
2        Dimethyl-Diazene  49.842235  50.442259  50.830463  51.122739   
3        Teramethylethene  78.154102  78.820593  79.307206  79.694611   
4                Isoprene  57.078311  58.159897  58.866675  59.400844   
5                 Azulene  34.193630  34.979860  35.471259  35.835997   
6          Trans-Stilbene  49.599450  50.523224  51.178233  51.692359   
7                Biphenyl  70.618601  71.725025  72.417923  72.932235   
8            Cis-Stilbene  57.012727  58.305150  59.109518  59.705579   
9                 Styrene  60.386257  61.709569  62.533935  63.145270   
10             Norbornene  73.647494  74.531964  75.143907  75.618603   
11                 Indene  59.306041  60.611450  61.444200  62.068129   
12           Cyclopentene  77.498981  78.714073  79